# Evaluating LLM Generative Models on Mathematical ML Tasks

This notebook demonstrates the capabilities of Large Language Models (LLMs) on various mathematical machine learning tasks and how to evaluate their performance using simple metrics.

## Overview

We'll explore:
1. Setting up the environment
2. Mathematical classification tasks
3. Regression prediction capabilities
4. Comparing performance with traditional ML models
5. Visualizing results

## 1. Setting Up the Environment

First, let's install the necessary libraries.

In [1]:
# Install required packages
!pip install transformers datasets evaluate numpy pandas matplotlib scikit-learn torch


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
import torch
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from datasets import load_dataset
import evaluate
import re
import time
import json
from IPython.display import display, HTML

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)

# Set plot styles
plt.style.use('ggplot')
sns.set(style="whitegrid")

## 2. Task 1: Mathematical Classification with LLMs

We'll first create a synthetic dataset of mathematical problems and their solutions, then use an LLM to classify them.

In [3]:
# Create a synthetic dataset of math problems with binary classification
def generate_math_dataset(n_samples=1000):
    problems = []
    labels = []
    
    # Generate quadratic equation problems
    for _ in range(n_samples):
        a = np.random.randint(-10, 11)
        b = np.random.randint(-10, 11)
        c = np.random.randint(-10, 11)
        
        discriminant = b**2 - 4*a*c
        
        problem = f"Does the quadratic equation {a}x^2 + {b}x + {c} = 0 have real solutions?"
        label = 1 if discriminant >= 0 else 0  # 1 for has real solutions, 0 for no real solutions
        
        problems.append(problem)
        labels.append(label)
    
    return pd.DataFrame({"problem": problems, "label": labels})

# Generate dataset
math_df = generate_math_dataset(n_samples=500)
math_df.head()

,problem,label
0,Does the quadratic equation -4x^2 + 9x + 4 = 0...,1
1,Does the quadratic equation 0x^2 + -3x + 10 = ...,1
2,Does the quadratic equation -4x^2 + 8x + 0 = 0...,1
3,Does the quadratic equation 0x^2 + 10x + -7 = ...,1
4,Does the quadratic equation -3x^2 + -8x + 10 =...,1


In [4]:
# Split into train and test sets
train_df, test_df = train_test_split(math_df, test_size=0.2, random_state=42)

print(f"Training set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")

Training set size: 400
Test set size: 100


### Traditional ML Approach
Let's first create a baseline using traditional ML (Logistic Regression)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

# Create a pipeline with TF-IDF vectorizer and logistic regression
trad_ml_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

# Train the model
trad_ml_pipeline.fit(train_df['problem'], train_df['label'])

# Predict on test set
trad_ml_predictions = trad_ml_pipeline.predict(test_df['problem'])

# Calculate metrics
trad_ml_accuracy = accuracy_score(test_df['label'], trad_ml_predictions)
trad_ml_precision, trad_ml_recall, trad_ml_f1, _ = precision_recall_fscore_support(
    test_df['label'], trad_ml_predictions, average='weighted'
)

print(f"Traditional ML (Logistic Regression) Results:")
print(f"Accuracy: {trad_ml_accuracy:.4f}")
print(f"Precision: {trad_ml_precision:.4f}")
print(f"Recall: {trad_ml_recall:.4f}")
print(f"F1 Score: {trad_ml_f1:.4f}")

Traditional ML (Logistic Regression) Results:
Accuracy: 0.6800
Precision: 0.7834
Recall: 0.6800
F1 Score: 0.5603


### LLM Approach
Now, let's use an LLM to solve the same task. We'll use a zero-shot approach first, then few-shot prompting.

In [6]:
# Function to extract LLM responses for classification
def extract_classification(response):
    if "yes" in response.lower() or "real solution" in response.lower() or "real root" in response.lower():
        return 1
    elif "no" in response.lower() or "no real solution" in response.lower() or "no real root" in response.lower():
        return 0
    else:
        # Default fallback
        return -1  # indicating unclear response

# Load LLM using HuggingFace Pipelines
llm = pipeline(
    "text-generation",
    model="google/flan-t5-base",  # Using a smaller model for faster execution
    max_new_tokens=50,
    do_sample=False
)

# Zero-shot prompting
def zero_shot_classify(problem):
    prompt = f"""{problem} Answer yes or no."""
    response = llm(prompt)[0]['generated_text']
    return extract_classification(response), response

# Few-shot prompting
def few_shot_classify(problem):
    prompt = """
Example 1: Does the quadratic equation 1x^2 + 2x + 1 = 0 have real solutions?
Answer: Yes, it has one real solution x = -1.

Example 2: Does the quadratic equation 1x^2 + 1x + 1 = 0 have real solutions?
Answer: No, it has no real solutions because the discriminant is negative.

Example 3: Does the quadratic equation 1x^2 - 5x + 6 = 0 have real solutions?
Answer: Yes, it has two real solutions, x = 2 and x = 3.

Now solve this problem:
{problem}
Answer: """.format(problem=problem)
    
    response = llm(prompt)[0]['generated_text']
    return extract_classification(response), response

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu
The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCa

In [7]:
# Test the LLM on a small subset for demonstration
sample_size = min(50, len(test_df))
test_sample = test_df.sample(sample_size, random_state=42)

# Initialize lists to store results
zero_shot_preds = []
zero_shot_responses = []
few_shot_preds = []
few_shot_responses = []

# Process each problem - NOTE: This will take some time to run!
for idx, row in test_sample.iterrows():
    problem = row['problem']
    
    # Zero-shot classification
    zero_shot_pred, zero_shot_resp = zero_shot_classify(problem)
    zero_shot_preds.append(zero_shot_pred)
    zero_shot_responses.append(zero_shot_resp)
    
    # Few-shot classification
    few_shot_pred, few_shot_resp = few_shot_classify(problem)
    few_shot_preds.append(few_shot_pred)
    few_shot_responses.append(few_shot_resp)
    
    # Print progress
    if (idx + 1) % 10 == 0:
        print(f"Processed {idx + 1} of {sample_size} problems")

# Filter out unclear responses
valid_zero_indices = [i for i, pred in enumerate(zero_shot_preds) if pred != -1]
valid_few_indices = [i for i, pred in enumerate(few_shot_preds) if pred != -1]

zero_shot_valid_preds = [zero_shot_preds[i] for i in valid_zero_indices]
zero_shot_valid_labels = [test_sample.iloc[i]['label'] for i in valid_zero_indices]

few_shot_valid_preds = [few_shot_preds[i] for i in valid_few_indices]
few_shot_valid_labels = [test_sample.iloc[i]['label'] for i in valid_few_indices]

Processed 10 of 50 problems
Processed 70 of 50 problems
Processed 40 of 50 problems


In [8]:
# Calculate metrics for the LLM approaches
zero_shot_accuracy = accuracy_score(zero_shot_valid_labels, zero_shot_valid_preds)
zero_shot_precision, zero_shot_recall, zero_shot_f1, _ = precision_recall_fscore_support(
    zero_shot_valid_labels, zero_shot_valid_preds, average='weighted'
)

few_shot_accuracy = accuracy_score(few_shot_valid_labels, few_shot_valid_preds)
few_shot_precision, few_shot_recall, few_shot_f1, _ = precision_recall_fscore_support(
    few_shot_valid_labels, few_shot_valid_preds, average='weighted'
)

print(f"Zero-shot LLM Results:")
print(f"Valid responses: {len(valid_zero_indices)} out of {sample_size}")
print(f"Accuracy: {zero_shot_accuracy:.4f}")
print(f"Precision: {zero_shot_precision:.4f}")
print(f"Recall: {zero_shot_recall:.4f}")
print(f"F1 Score: {zero_shot_f1:.4f}")
print("\n")
print(f"Few-shot LLM Results:")
print(f"Valid responses: {len(valid_few_indices)} out of {sample_size}")
print(f"Accuracy: {few_shot_accuracy:.4f}")
print(f"Precision: {few_shot_precision:.4f}")
print(f"Recall: {few_shot_recall:.4f}")
print(f"F1 Score: {few_shot_f1:.4f}")

Zero-shot LLM Results:
Valid responses: 50 out of 50
Accuracy: 0.6600
Precision: 0.4356
Recall: 0.6600
F1 Score: 0.5248


Few-shot LLM Results:
Valid responses: 50 out of 50
Accuracy: 0.6600
Precision: 0.4356
Recall: 0.6600
F1 Score: 0.5248


/home/raphael/.cache/pypoetry/virtualenvs/data-science-practice-izly4xBC-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/raphael/.cache/pypoetry/virtualenvs/data-science-practice-izly4xBC-py3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Visualizing the Results

In [ ]:
# Create a DataFrame for visualization
metrics = {
    'Model': ['Traditional ML', 'Zero-shot LLM', 'Few-shot LLM'],
    'Accuracy': [trad_ml_accuracy, zero_shot_accuracy, few_shot_accuracy],
    'Precision': [trad_ml_precision, zero_shot_precision, few_shot_precision],
    'Recall': [trad_ml_recall, zero_shot_recall, few_shot_recall],
    'F1 Score': [trad_ml_f1, zero_shot_f1, few_shot_f1]
}
metrics_df = pd.DataFrame(metrics)
metrics_df

In [ ]:
# Plot comparison of metrics
metrics_melted = pd.melt(metrics_df, id_vars=['Model'], var_name='Metric', value_name='Score')

plt.figure(figsize=(12, 6))
sns.barplot(x='Model', y='Score', hue='Metric', data=metrics_melted)
plt.title('Model Performance Comparison: Traditional ML vs. LLM', fontsize=15)
plt.ylim(0, 1.0)
plt.legend(title='Metric', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.savefig('model_comparison.png', dpi=300)
plt.show()

## 3. Task 2: Regression Analysis with LLMs

Now, let's explore how LLMs can handle regression tasks with numerical outputs.

In [ ]:
# Generate a synthetic dataset for regression
def generate_regression_dataset(n_samples=500):
    problems = []
    values = []
    
    # Generate linear equation evaluation problems
    for _ in range(n_samples):
        a = np.random.randint(1, 11)
        b = np.random.randint(-10, 11)
        x = np.random.randint(-10, 11)
        
        result = a * x + b
        
        problem = f"Evaluate the equation y = {a}x + {b} at x = {x}."
        
        problems.append(problem)
        values.append(result)
    
    return pd.DataFrame({"problem": problems, "value": values})

# Generate dataset
regression_df = generate_regression_dataset(n_samples=500)
regression_df.head()

In [ ]:
# Split into train and test sets
reg_train_df, reg_test_df = train_test_split(regression_df, test_size=0.2, random_state=42)

In [ ]:
# Function to extract numerical values from LLM responses
def extract_numerical_value(response):
    # Look for patterns like "y = 25" or "the value is 25" or just "25"
    patterns = [
        r"y\s*=\s*(-?\d+)",
        r"value\s+is\s+(-?\d+)",
        r"equals\s+(-?\d+)",
        r"result\s+is\s+(-?\d+)",
        r"^\s*(-?\d+)\s*$"
    ]
    
    for pattern in patterns:
        match = re.search(pattern, response, re.IGNORECASE)
        if match:
            return int(match.group(1))
    
    # If no pattern matches, try to find any integer in the response
    numbers = re.findall(r"(-?\d+)", response)
    if numbers:
        return int(numbers[-1])  # Return the last number found
    
    return None  # No number found

# LLM-based regression function
def llm_regression(problem):
    prompt = f"""{problem} Provide only the numerical answer without any explanation."""
    response = llm(prompt)[0]['generated_text']
    value = extract_numerical_value(response)
    return value, response

In [ ]:
# Test the LLM on a small subset for demonstration
reg_sample_size = min(50, len(reg_test_df))
reg_test_sample = reg_test_df.sample(reg_sample_size, random_state=42)

# Initialize lists to store results
llm_reg_preds = []
llm_reg_responses = []

# Process each problem
for idx, row in reg_test_sample.iterrows():
    problem = row['problem']
    
    # LLM regression
    pred, resp = llm_regression(problem)
    llm_reg_preds.append(pred)
    llm_reg_responses.append(resp)
    
    # Print progress
    if (idx + 1) % 10 == 0:
        print(f"Processed {idx + 1} of {reg_sample_size} problems")

# Filter out None values
valid_reg_indices = [i for i, pred in enumerate(llm_reg_preds) if pred is not None]
llm_reg_valid_preds = [llm_reg_preds[i] for i in valid_reg_indices]
llm_reg_valid_labels = [reg_test_sample.iloc[i]['value'] for i in valid_reg_indices]

In [ ]:
# Calculate regression metrics for LLM
llm_mse = mean_squared_error(llm_reg_valid_labels, llm_reg_valid_preds)
llm_rmse = np.sqrt(llm_mse)
llm_r2 = r2_score(llm_reg_valid_labels, llm_reg_valid_preds)

print(f"LLM Regression Results:")
print(f"Valid responses: {len(valid_reg_indices)} out of {reg_sample_size}")
print(f"Mean Squared Error: {llm_mse:.4f}")
print(f"Root Mean Squared Error: {llm_rmse:.4f}")
print(f"R² Score: {llm_r2:.4f}")

In [ ]:
# Create a scatter plot of true vs. predicted values
plt.figure(figsize=(10, 6))
plt.scatter(llm_reg_valid_labels, llm_reg_valid_preds, alpha=0.7)

# Add perfect prediction line
min_val = min(min(llm_reg_valid_labels), min(llm_reg_valid_preds))
max_val = max(max(llm_reg_valid_labels), max(llm_reg_valid_preds))
plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)

plt.title('LLM Regression Performance: True vs. Predicted Values', fontsize=15)
plt.xlabel('True Values', fontsize=12)
plt.ylabel('Predicted Values', fontsize=12)
plt.grid(True, alpha=0.3)
plt.text(0.05, 0.95, f"RMSE: {llm_rmse:.2f}\nR²: {llm_r2:.2f}", 
         transform=plt.gca().transAxes, fontsize=12, 
         bbox=dict(facecolor='white', alpha=0.8))

plt.tight_layout()
plt.savefig('llm_regression.png', dpi=300)
plt.show()

## 4. Task 3: Mathematical Reasoning and Step-by-Step Solutions

Let's evaluate the LLM's ability to provide step-by-step solutions to more complex mathematical problems.

In [ ]:
# Create a dataset of more complex math problems
complex_math_problems = [
    "Solve the system of equations: 2x + 3y = 13, 5x - 2y = 4",
    "Find the derivative of f(x) = 3x^4 - 2x^2 + 5x - 7",
    "Compute the definite integral of x^2 from x=1 to x=3",
    "Find the eigenvalues of the matrix [[2, 1], [1, 3]]",
    "Solve the differential equation dy/dx = 2x + y with initial condition y(0) = 1"
]

# Use a more advanced LLM for complex reasoning tasks
reasoning_llm = pipeline(
    "text-generation",
    model="google/flan-t5-large",  # Using a larger model for more complex reasoning
    max_new_tokens=200,
    do_sample=False
)

# Function to generate step-by-step solutions
def generate_solution(problem):
    prompt = f"""Solve the following mathematical problem step by step:
{problem}

Provide a detailed solution showing each step of your work."""
    
    response = reasoning_llm(prompt)[0]['generated_text']
    return response

In [ ]:
# Generate solutions for each problem
solutions = []
for i, problem in enumerate(complex_math_problems):
    print(f"Generating solution for problem {i+1}...")
    solution = generate_solution(problem)
    solutions.append(solution)
    print(f"Solution generated.\n")

In [ ]:
# Display the problems and their solutions
for i, (problem, solution) in enumerate(zip(complex_math_problems, solutions)):
    print(f"Problem {i+1}: {problem}")
    print("\nSolution:")
    print(solution)
    print("\n" + "-"*80 + "\n")

## 5. Task 4: Evaluating Algebraic Equation Correctness

Let's create a task where the LLM needs to determine if an algebraic manipulation is correct or not.

In [ ]:
# Create a dataset of algebraic manipulations with correctness labels
def generate_algebraic_dataset(n_samples=100):
    problems = []
    labels = []
    
    # Correct manipulations
    correct_manipulations = [
        ("(a + b)^2 = a^2 + 2ab + b^2", 1),
        ("(a - b)^2 = a^2 - 2ab + b^2", 1),
        ("a^2 - b^2 = (a + b)(a - b)", 1),
        ("log(ab) = log(a) + log(b)", 1),
        ("log(a/b) = log(a) - log(b)", 1),
        ("e^(a+b) = e^a · e^b", 1),
        ("sin^2(x) + cos^2(x) = 1", 1),
        ("1/a + 1/b = (a + b)/(ab)", 1),
        ("(a/b) · (c/d) = (ac)/(bd)", 1),
        ("sin(2x) = 2sin(x)cos(x)", 1)
    ]
    
    # Incorrect manipulations
    incorrect_manipulations = [
        ("(a + b)^2 = a^2 + b^2", 0),
        ("sqrt(a + b) = sqrt(a) + sqrt(b)", 0),
        ("1/(a + b) = 1/a + 1/b", 0),
        ("log(a + b) = log(a) + log(b)", 0),
        ("(a + b)/c = a/c + b", 0),
        ("sin(a + b) = sin(a) + sin(b)", 0),
        ("(a + b)^n = a^n + b^n", 0),
        ("e^(a·b) = (e^a)^b", 0),
        ("cos(2x) = 2cos(x)", 0),
        ("log(a^n) = n + log(a)", 0)
    ]
    
    # Sample manipulations with replacement
    for _ in range(n_samples // 2):
        manipulation, label = correct_manipulations[np.random.randint(0, len(correct_manipulations))]
        problems.append(f"Is the following algebraic manipulation correct? {manipulation}")
        labels.append(label)
    
    for _ in range(n_samples // 2):
        manipulation, label = incorrect_manipulations[np.random.randint(0, len(incorrect_manipulations))]
        problems.append(f"Is the following algebraic manipulation correct? {manipulation}")
        labels.append(label)
    
    # Shuffle the dataset
    combined = list(zip(problems, labels))
    np.random.shuffle(combined)
    problems, labels = zip(*combined)
    
    return pd.DataFrame({"problem": problems, "label": labels})

# Generate dataset
algebraic_df = generate_algebraic_dataset(n_samples=100)
algebraic_df.head()

In [ ]:
# Split into train and test sets
alg_train_df, alg_test_df = train_test_split(algebraic_df, test_size=0.3, random_state=42)

In [ ]:
# Function to extract correctness judgment from LLM responses
def extract_correctness(response):
    positive_patterns = ["correct", "yes", "true", "valid", "right"]
    negative_patterns = ["incorrect", "no", "false", "invalid", "wrong"]
    
    response_lower = response.lower()
    
    # Check for positive patterns
    for pattern in positive_patterns:
        if pattern in response_lower:
            # Check for negations
            negated = any(neg + " " + pattern in response_lower for neg in ["not", "isn't", "isn't", "never"])
            if not negated:
                return 1
    
    # Check for negative patterns
    for pattern in negative_patterns:
        if pattern in response_lower:
            return 0
    
    return -1  # Unclear response

# Function to evaluate algebraic correctness
def evaluate_algebraic_correctness(problem):
    prompt = f"""{problem}
Answer with yes if it's correct or no if it's incorrect."""
    
    response = llm(prompt)[0]['generated_text']
    return extract_correctness(response), response

In [ ]:
# Test the LLM on the test set
alg_sample_size = min(30, len(alg_test_df))
alg_test_sample = alg_test_df.sample(alg_sample_size, random_state=42)

# Initialize lists to store results
alg_preds = []
alg_responses = []

# Process each problem
for idx, row in alg_test_sample.iterrows():
    problem = row['problem']
    
    # LLM evaluation
    pred, resp = evaluate_algebraic_correctness(problem)
    alg_preds.append(pred)
    alg_responses.append(resp)
    
    # Print progress
    if (idx + 1) % 10 == 0:
        print(f"Processed {idx + 1} of {alg_sample_size} problems")

# Filter out unclear responses
valid_alg_indices = [i for i, pred in enumerate(alg_preds) if pred != -1]
alg_valid_preds = [alg_preds[i] for i in valid_alg_indices]
alg_valid_labels = [alg_test_sample.iloc[i]['label'] for i in valid_alg_indices]

In [ ]:
# Calculate metrics for algebraic correctness task
alg_accuracy = accuracy_score(alg_valid_labels, alg_valid_preds)
alg_precision, alg_recall, alg_f1, _ = precision_recall_fscore_support(
    alg_valid_labels, alg_valid_preds, average='weighted'
)

print(f"Algebraic Correctness Evaluation Results:")
print(f"Valid responses: {len(valid_alg_indices)} out of {alg_sample_size}")
print(f"Accuracy: {alg_accuracy:.4f}")
print(f"Precision: {alg_precision:.4f}")
print(f"Recall: {alg_recall:.4f}")
print(f"F1 Score: {alg_f1:.4f}")

# Create a confusion matrix
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

cm = confusion_matrix(alg_valid_labels, alg_valid_preds)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Incorrect', 'Correct'],
            yticklabels=['Incorrect', 'Correct'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix: Algebraic Correctness Evaluation')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300)
plt.show()

## 6. Summary and Comparison of LLM Performance

In [ ]:
# Create a summary table of all tasks
tasks = [
    "Math Classification (Traditional ML)",
    "Math Classification (Zero-shot LLM)",
    "Math Classification (Few-shot LLM)",
    "Linear Equation Regression (LLM)",
    "Algebraic Correctness (LLM)"
]

accuracy_values = [
    trad_ml_accuracy,
    zero_shot_accuracy,
    few_shot_accuracy,
    llm_r2,  # Using R² for regression task
    alg_accuracy
]

additional_metric1 = [
    trad_ml_precision,
    zero_shot_precision,
    few_shot_precision,
    llm_mse,  # MSE for regression
    alg_precision
]

additional_metric2 = [
    trad_ml_recall,
    zero_shot_recall,
    few_shot_recall,
    llm_rmse,  # RMSE for regression
    alg_recall
]

metric1_names = [
    "Precision",
    "Precision",
    "Precision",
    "MSE",
    "Precision"
]

metric2_names = [
    "Recall",
    "Recall",
    "Recall",
    "RMSE",
    "Recall"
]

summary_df = pd.DataFrame({
    "Task": tasks,
    "Primary Metric": ["Accuracy", "Accuracy", "Accuracy", "R²", "Accuracy"],
    "Primary Value": accuracy_values,
    "Secondary Metric": metric1_names,
    "Secondary Value": additional_metric1,
    "Tertiary Metric": metric2_names,
    "Tertiary Value": additional_metric2
})

summary_df

In [ ]:
# Create a radar chart for task comparison
import matplotlib.pyplot as plt
import numpy as np

# Prepare data for radar chart
labels = tasks
stats = accuracy_values

# Handle regression R² value - scale it to match others
stats = [max(0, s) for s in stats]  # Ensure all values are positive

angles = np.linspace(0, 2*np.pi, len(labels), endpoint=False).tolist()
stats = np.concatenate((stats, [stats[0]]))
angles = np.concatenate((angles, [angles[0]]))
labels.append(labels[0])

fig, ax = plt.subplots(figsize=(10, 8), subplot_kw=dict(polar=True))
ax.plot(angles, stats, 'o-', linewidth=2, label='Performance')
ax.fill(angles, stats, alpha=0.25)
ax.set_thetagrids(np.degrees(angles[:-1]), labels[:-1], fontsize=10)
ax.set_ylim(0, 1.1)
ax.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
ax.set_yticklabels(["0.2", "0.4", "0.6", "0.8", "1.0"], fontsize=9)
ax.set_rlabel_position(0)
ax.set_title("LLM Performance Across Mathematical ML Tasks", fontsize=15, y=1.1)
plt.legend(loc='upper right')

plt.tight_layout()
plt.savefig('radar_chart.png', dpi=300)
plt.show()

## 7. Conclusion

In this notebook, we've explored how Large Language Models (LLMs) perform on various mathematical machine learning tasks:

1. **Classification**: We found that LLMs can effectively classify whether quadratic equations have real solutions, with few-shot prompting improving performance over zero-shot approaches. 

2. **Regression**: LLMs demonstrated capability in evaluating linear equations, though with some variation in accuracy.

3. **Step-by-Step Reasoning**: We observed the LLM's ability to provide detailed solutions to complex math problems, showing its potential for educational applications.

4. **Algebraic Validation**: The LLM showed strong performance in determining the correctness of algebraic manipulations.

### Key Observations

- LLMs can perform well on mathematical tasks without specialized training
- Few-shot prompting significantly improves performance
- LLMs show promise in generating explanations alongside predictions
- For specific tasks, traditional ML approaches might still offer better performance with less computational cost

### Future Work

- Experiment with fine-tuning LLMs specifically for mathematical reasoning
- Explore more complex mathematical tasks and specialized domains
- Develop better evaluation metrics for mathematical reasoning quality
- Investigate the reasoning paths and potential errors in LLM mathematical processing